In [1]:
import pickle
from natsort import natsorted

basedir = '../../ImageNomer/data/anton/cohorts/PNC'
demoname = f'{basedir}/demographics.pkl'

with open(demoname, 'rb') as f:
    demo = pickle.load(f)
    
subs = natsorted(list(demo['age_at_cnb'].keys()))
print(len(subs))

1529


In [22]:
import numpy as np

basis_file = '/home/anton/Documents/Tulane/Research/PNC_Good/AngleBasis5.pkl'
basis = pickle.load(open(basis_file, 'rb'))

# FC, age, sex, race, thetas, jitter
rest = [[],[],[],[],[],[]]
nback = [[],[],[],[],[],[]]
emoid = [[],[],[],[],[],[]]

for sub in subs:
    for task, struct in zip(['rest', 'nback', 'emoid'], [rest, nback, emoid]):
        try:
            if demo['Race'][sub] not in ['AA', 'EA']:
                continue
            p = np.load(f'{basedir}/fc/{sub}_task-{task}_fc.npy')
            a = demo['age_at_cnb'][sub]
            s = demo['Sex'][sub] == 'M'
            r = demo['Race'][sub] == 'AA'
            ab = basis[f'{sub}-{task}']
            struct[1].append(a)
            struct[2].append(s)
            struct[3].append(r)
            struct[0].append(p)
            struct[4].append(ab['thetas'])
            struct[5].append(ab['jitter'])
        except:
            continue
    
for struct in [rest, nback, emoid]:
    struct[0] = np.stack(struct[0])
    struct[1] = np.array(struct[1])
    struct[2] = np.array(struct[2]).astype('int')
    struct[3] = np.array(struct[3]).astype('int')
    struct[4] = np.stack(struct[4])
    struct[5] = np.stack(struct[5])
    
    print([a.shape for a in struct])


[(1230, 34716), (1230,), (1230,), (1230,), (1230, 5, 264), (1230, 5, 264)]
[(1290, 34716), (1290,), (1290,), (1290,), (1290, 5, 264), (1290, 5, 264)]
[(1329, 34716), (1329,), (1329,), (1329,), (1329, 5, 264), (1329, 5, 264)]


In [23]:
def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def tops(thetas, jitter):
    t0 = np.expand_dims(thetas, 2)
    t1 = np.expand_dims(thetas, 3)
    j0 = np.expand_dims(jitter, 2)
    j1 = np.expand_dims(jitter, 3)
    ps = np.cos(t0-t1)*(j0*j1)
    a,b = np.triu_indices(264, 1)
    ps = ps[:,:,a,b]
    return ps
    
for st in [rest, nback, emoid]:
    ps = tops(st[4], st[5])
    aps = np.mean(ps, axis=1)
    res = st[0] - aps

    print(ps.shape)
    print(aps.shape)
    print(res.shape)
    
    st.append(aps)
    st.append(res)
    print(len(st))

(1230, 5, 34716)
(1230, 34716)
(1230, 34716)
8
(1290, 5, 34716)
(1290, 34716)
(1290, 34716)
8
(1329, 5, 34716)
(1329, 34716)
(1329, 34716)
8


In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def cat(x, **kwargs):
    return np.concatenate(x, **kwargs)

accs0, accs1, accs2 = [], [], []

mod1 = emoid
mod2 = nback

for _ in range(10):
    x = cat([mod1[7], mod2[7]], axis=0)
    z = cat([mod1[6], mod2[6]], axis=0)
    w = cat([mod1[0], mod2[0]], axis=0)
    y = cat([np.zeros(mod1[7].shape[0]), np.ones(mod2[7].shape[0])], axis=0)

    xtr, xt, ztr, zt, wtr, wt, ytr, yt = train_test_split(x, z, w, y, stratify=y, train_size=0.5)

    reg = LogisticRegression(C=1, max_iter=1000).fit(xtr, ytr)
    yhat = reg.predict(xt)
    acc = np.mean(yhat == yt)
    print(acc)
    accs0.append(acc)
    
    reg = LogisticRegression(C=1, max_iter=1000).fit(ztr, ytr)
    yhat = reg.predict(zt)
    acc = np.mean(yhat == yt)
    print(acc)
    accs1.append(acc)
    
    reg = LogisticRegression(C=1, max_iter=1000).fit(wtr, ytr)
    yhat = reg.predict(wt)
    acc = np.mean(yhat == yt)
    print(acc)
    accs2.append(acc)
    
print('---')
for a in [accs0, accs1, accs2]:
    print(np.mean(a), np.std(a))

0.9374045801526718
0.8770992366412214
0.8961832061068702
0.932824427480916
0.8900763358778626
0.9106870229007633
0.9412213740458015
0.8969465648854962
0.9129770992366413
0.932824427480916
0.8755725190839695
0.9083969465648855
0.932824427480916
0.8893129770992366
0.9061068702290076
0.9412213740458015
0.8778625954198473
0.9061068702290076
0.932824427480916
0.8854961832061069
0.9053435114503817
0.934351145038168
0.8824427480916031
0.9053435114503817
0.9412213740458015
0.8870229007633588
0.9076335877862596
0.9358778625954198
0.8824427480916031
0.9061068702290076
---
0.9362595419847329 0.003551874847063177
0.8844274809160307 0.0063519523855617415
0.9064885496183207 0.00417062240599932
